In [33]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [34]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/alexey/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['source', 'show', 'figure', 'text']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
responses = []
with bz2.BZ2File('../5.3/banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [03:18, 1013.94it/s]


#  Домашнее задание по NLP # 2 [100 баллов] 
## Составление словарей для классификации по тональности
При классификации текстов или предложений по тональности необходимо использовать оценочные словари для предметной области, то есть, такие словари, в которых содержатся отрицательные и позитивные слова для какой-то предметной области. Идея подобных словарей основана на следующих наблюдениях: во-первых, для разных товаров используются разные оценочные слова (например бывает “захватывающая книга”, но не бывает “захватывающих лыж”), во-вторых, в контексте разных товаров одни и те же слова могут иметь разную окраску (слово “тормоз” в отзыве на велосипед имеет нейтральную окраску, в отзыве на компьютер – резко негативную, “пыль” в контексте пылесосов – нейтральную, в контексте кофемолок – положительную (“мелкий помол в пыль”)). Еще один пример: "теплое пиво" – это плохо, а "теплый свитер" – это хорошо.  

Составление таких словарей в ручную – трудоемкий процесс, но, к счастью, его не сложно автоматизировать, если собрать достаточно большие корпуса отзывов. В этом домашнем задании вам предстоит попробовать реализовать один их подходов к составлению оценочных словарей, основанный на статье Inducing Domain-Specific Sentiment Lexicons from Unlabeled Corpora (https://nlp.stanford.edu/pubs/hamilton2016inducing.pdf).


Данные для задания – уже знакомые вам отзывы на банки, собранные с нескольких сайтов Рунета. Отзывы могут быть как положительными (оценка 5), так и отрицательными (оценка 1).

In [5]:
responses[99]

{'author': 'ronnichka',
 'bank_license': 'лицензия № 880',
 'bank_name': 'Югра',
 'city': 'г. Саратов',
 'datetime': '2015-06-03 20:56:57',
 'num_comments': 0,
 'rating_grade': 3,
 'rating_not_checked': False,
 'text': 'Здравствуйте! Хотела написать, что мне месяц не выдают карту ко вкладу, ссылаясь на "нам же их из Самары везут" (на секундочку 5 часов езды от нашего города). Но! Прочитала, что людям 3,5 месяцев не выдают карту, и поняла, что у меня все хорошо, пока что. И подарок мне дали, и кулер в отделении есть. Так что я, конечно, готова ждать. Правда хотелось бы не очень долго.',
 'title': 'Карта ко вкладу'}

## Часть 1. Обучение модели word2vec [50 баллов]

1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова. 
2. Обучите по коллекции предложений word2vec
3. Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    * тест на определение ближайших слов
    * тест на аналогии (мужчина – король : женщина – королева)
    * тест на определение лишнего слова.
    
4. Постройте несколько визуализаций:
    * TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
    * задайте координаты для нового пространства следующим образом: одна  ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах.  Более формально:
    берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.  
 


Ссылка на примеры визуализаций: https://towardsdatascience.com/game-of-thrones-word-embeddings-does-r-l-j-part-2-30290b1c0b4b

In [6]:
from pymystem3 import Mystem
import re

m = Mystem()

regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [7]:
#Импортируем имеющийся корпус стоп-слов из nltk
from nltk.corpus import stopwords

#Добавляем несколько своих стои-слов
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д', '-']

#Оописываем функцию удаления стоп-слов
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

In [8]:
#Разбиваем текст на слова, слова приводим к нижнему регистру, лематизируем и выбрасываем стпо-слова
fulltext = []
for response in tqdm(responses):
    
    #Приведем все слова к нижнему регистру
    r=response['text'].lower()
    #Оставляем только слова, в соответствии с шаблоном
    r=words_only(r)
    #Лематизируем слова
    r=lemmatize(r)
    #Удаляем стоп-слова
    r=remove_stopwords(r)
    #Добавляем новый отчет в список
    fulltext.append(r.split())

100%|██████████| 153499/153499 [1:02:31<00:00, 40.91it/s]


## Обучение модели в gensim

In [9]:
%%time
#Обучим модель word2vec от gensim
from gensim.models import Word2Vec
model = Word2Vec(fulltext, size=100, window=5, min_count=5, workers=4)
#На всякий случай, сохраняем модель в файл
model.save("sent_w2v.model")

CPU times: user 5min 31s, sys: 8.21 s, total: 5min 40s
Wall time: 3min 4s


In [ ]:
#Загрузка данных из файла (на всякий случай)
#from gensim.models import Word2Vec
#model = Word2Vec.load("sent_w2v.model") 

In [10]:
#Посмотрим самые близкие слова для смайлика
model.most_similar(":)")

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('хороший!', 0.6438833475112915),
 ('супер!', 0.6353523135185242),
 (':d', 0.6267712712287903),
 ('удача!', 0.6261351704597473),
 ('эх', 0.6208802461624146),
 ('удача', 0.6168009638786316),
 ('держать!', 0.6101917624473572),
 ('позади', 0.6094632744789124),
 ('молодец!', 0.6053237915039062),
 ('больно', 0.6038185954093933)]

In [ ]:
#Для смайлика самым близким словом подобралось "хороший", что вполне корректно

In [11]:
model.most_similar("банк")

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('банка', 0.8490789532661438),
 ('банк!', 0.708834171295166),
 ('банк)', 0.5941247344017029),
 ('банк:', 0.5733256936073303),
 ('сбербанк', 0.5622051954269409),
 ('ситибанк', 0.5366489887237549),
 ('мкб', 0.5184993743896484),
 ('хотя', 0.5088075995445251),
 ('росбанк', 0.502704381942749),
 ('сбер', 0.4618588089942932)]

In [ ]:
#Самыми близкими оказались различные колллизии слова банк

In [12]:
model.most_similar("король")

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('овца', 0.7836336493492126),
 ('гопник', 0.7746967077255249),
 ('курица', 0.7661190032958984),
 ('талант', 0.7531870603561401),
 ('снобизм', 0.7524035573005676),
 ('совсем!', 0.7499550580978394),
 ('муравей', 0.7464995384216309),
 ('мальчишка', 0.744761049747467),
 ('инфантильный', 0.7400878071784973),
 ('стадо', 0.7335176467895508)]

In [ ]:
#В данном контексте, прямо как пример изменения смысла слова. В отзывах о банках слово король имеет явно негативный оттенок
#и употребляется скорее в саркостическом смысле

In [13]:
model.most_similar("королева")

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('балашиха', 0.8839225769042969),
 ('серпухов', 0.8834270238876343),
 ('электросталь', 0.8756070733070374),
 ('красногорск', 0.8748654723167419),
 ('мытищи', 0.8720663785934448),
 ('люберцы', 0.870891809463501),
 ('реутов', 0.869674801826477),
 ('космонавт', 0.8644417524337769),
 ('сыктывкар', 0.8644206523895264),
 ('долгопрудный', 0.8632749915122986)]

In [ ]:
#Видимо королеву считают городом Королев и ищутся близкие слова

In [14]:
model.most_similar("карта")

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('карточка', 0.8889328837394714),
 ('карта!', 0.8274955153465271),
 ('карта)', 0.7209641933441162),
 ('кредитка', 0.6868469715118408),
 ('карта:', 0.6789650917053223),
 ('неименной', 0.6392953991889954),
 ('именной', 0.6150289177894592),
 ('пластик', 0.6072357892990112),
 ('счет', 0.5781598687171936),
 ('дебетовый', 0.5726080536842346)]

In [ ]:
#В данном случае слова выглядят вполне разумно

In [28]:
model.most_similar(positive=["карта", "удобно"], negative=["отделение"])

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('кредитка', 0.6113500595092773),
 ('карточка', 0.5969126224517822),
 ('дебетовка', 0.5782869458198547),
 ('пластик', 0.5424693822860718),
 ('бесплатно', 0.5061665773391724),
 ('бвк', 0.49363696575164795),
 ('карта:', 0.4935533106327057),
 ('кукуруза', 0.4873521029949188),
 ('удобный', 0.4827096164226532),
 ('карта)', 0.4786989688873291)]

In [ ]:
#Честно говоря, ожидал, что в таком контексте самым близким словом будет слово "плохо"

In [26]:
model.most_similar(positive=["сбербанк","хороший"], negative=["тинькоф"])

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('сбер', 0.4700581133365631),
 ('сб', 0.45253700017929077),
 ('плохой', 0.4450751543045044),
 ('банк', 0.43362003564834595),
 ('банка', 0.39804112911224365),
 ('мкб', 0.39123862981796265),
 ('улучшаться', 0.3828800320625305),
 ('поменяться', 0.3811820447444916),
 ('главный', 0.37704917788505554),
 ('изменяться', 0.3757723271846771)]

In [16]:
model.most_similar(positive=["король","мужчина"], negative=["женщина"])

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('таракан', 0.7257843613624573),
 ('гопник', 0.7159368395805359),
 ('курица', 0.7104015946388245),
 ('овца', 0.7075490951538086),
 ('креативность', 0.7070243954658508),
 ('публика', 0.698686420917511),
 ('лопата', 0.6972745656967163),
 ('снобизм', 0.6949231028556824),
 ('талант', 0.6912837624549866),
 ('пафос', 0.6911239624023438)]

In [24]:
model.doesnt_match("борщ сметана макароны пирожок консомэ кошка".split())

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'пирожок'

In [23]:
model.doesnt_match("мкб сбербанк втб тинькоф карта".split())

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'карта'

In [ ]:
#Интересный момент. Если написать "Тиньков", то словарь, видимо не ассоциирует это слово с банком и выкидывает именно его

### Визуализация пространства слов 

In [18]:
top_words = []
from nltk import FreqDist

#Формируем словарь частотности слов
fd = FreqDist()
for text in fulltext:
    fd.update(text)

#Сохраняем 500 наиболее часто употребляемых
for i in fd.most_common(500):
    top_words.append(i[0])

#Выводим топ 500 слов на печать
print(top_words)

['банк', 'карта', 'деньги', 'день', 'кредит', 'который', 'весь', 'отделение', 'сотрудник', 'клиент', 'мочь', 'счет', 'свой', 'сказать', 'сумма', 'получать', 'заявление', 'год', 'время', 'вопрос', 'кредитный', 'приходить', 'вклад', 'офис', 'данный', 'телефон', 'ваш', 'платеж', 'договор', 'звонить', 'ответ', 'номер', ')', 'банкомат', 'сбербанк', 'очень', 'позвонить', 'написать', 'г', 'обращаться', 'месяц', 'решать', 'проблема', 'средство', 'работать', 'работа', 'рубль', 'документ', 'звонок', 'ситуация', 'отвечать', 'хотеть', 'почему', 'человек', 'операция', 'давать', 'говорить', 'информация', 'сообщать', 'сделать', 'оформлять', 'смс', 'срок', 'услуга', 'просто', 'должный', 'заявка', 'банка', 'знать', 'обслуживание', 'очередь', ':', 'сегодня', 'минута', 'первый', 'девушка', 'ждать', 'руб', 'сайт', 'претензия', 'новый', 'ничто', 'просить', 'интернет', 'открывать', 'принимать', 'снимать', 'процент', 'выдавать', 'никакой', 'являться', 'неделя', 'оператор', 'становиться', 'понимать', '%', 'пр

In [19]:
top_words_vec = model[top_words]

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [20]:
#При помощи TSNE понижаем размерность до 2-х компонент, чтобы можно было нарисовать пространство слов
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [21]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

In [ ]:
#На визуализации видно, что радом расположены слова "Снятие", "снимать", "вносить", "полагать", "переводить", "пополнять"
#"поступать", "зачисление", "зачислять", "начислять", "возврат" - все их можно отнести к кластеру Работа с банковской картой
#Другой кластер - "номер", "телефон", "колл", "телефонный", "трубка". Можно их обозначить как Обращения по телефону
#Еще один кластер - "тысяча", "рубль", "руб", "размер", "сумма". Можно их обозначить как Информация о деньгах

## Часть 2. Распространение метки [50 баллов]

Определите 5-8 позитивных слов (например, “быстрый”, “удобный”) и 5-8  негативных слов (например,“очередь”, “медленно”). Эти слова будут основной будущего оценочного словаря. Пусть позитивному классу соответствует метка 1, негативному – -1. Пометьте выбранные слова в лексическом графе соответствующими метками. Запустите любой известный вам метод распространения метки (Label Propogation) в лексическом графе. На выходе метода распространения ошибки должны быть новые слова, помеченные метками 1 и -1 – это и есть искомые оценочные слова.

Алгоритмы распространения метки устроены примерно так: пусть мы находимся в выршине, помеченном +1. С какой-то вероятностью мы переносим эту метку на соседние узлы. С меньшей вероятностью переносим ее на вершины на расстоянии два. В конце распространения метки, часть вершин оказывается помечена меткой +1, часть – -1, большая часть остается без метки.

Рекомендуемые алгоритмы распространения метки:
1. ```graphlab.label_propagation``` (```graphlab``` доступен бесплатно по образовательной лицензии)
2. ```sklearn.semi_supervised.LabelPropagation``` 
3. ```sklearn.semi_supervised.LabelSpreading```

In [40]:
# пример построения графа 

import jgraph as ig
g = ig.Graph(directed=True)
for word in model.wv.vocab.keys():
    g.add_vertex(word)
    
    
    
for word in model.wv.vocab.keys() :
    node = g.vs.select(name = word).indices[0]
    similar_words = model.most_similar(word, topn=5)
    for sim in similar_words:
        word1 = sim[0]
        val  = sim[1]
        new_node = g.vs.select(name = word1).indices[0]
        g.add_edge(node, new_node, weight = val)

AttributeError: module 'jgraph' has no attribute 'Graph'